In [1]:
import numpy

In [2]:
import pandas as pd

In [6]:
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpStatus

In [7]:
import sklearn

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [9]:
flights_data = pd.read_csv('Business_Dataset.csv')

/var/folders/qv/2tgf4xk15yl2x3t63qygm7y40000gn/T/ipykernel_82854/353090443.py:1: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_data = pd.read_csv('Business_Dataset.csv')


In [9]:
print(flights_data.head())

       tbl  Year  quarter  citymarketid_1  citymarketid_2  \
0  Table1a  2021        3           30135           33195   
1  Table1a  2021        3           30135           33195   
2  Table1a  2021        3           30140           30194   
3  Table1a  2021        3           30140           30194   
4  Table1a  2021        3           30140           30466   

                            city1                          city2  airportid_1  \
0  Allentown/Bethlehem/Easton, PA  Tampa, FL (Metropolitan Area)        10135   
1  Allentown/Bethlehem/Easton, PA  Tampa, FL (Metropolitan Area)        10135   
2                 Albuquerque, NM          Dallas/Fort Worth, TX        10140   
3                 Albuquerque, NM          Dallas/Fort Worth, TX        10140   
4                 Albuquerque, NM                    Phoenix, AZ        10140   

   airportid_2 airport_1  ...    fare  carrier_lg  large_ms  fare_lg  \
0        14112       ABE  ...   81.43          G4    1.0000    81.43   
1 

In [12]:
flights_data.columns

Index(['tbl', 'Year', 'quarter', 'citymarketid_1', 'citymarketid_2', 'city1',
       'city2', 'airportid_1', 'airportid_2', 'airport_1', 'airport_2',
       'nsmiles', 'passengers', 'fare', 'carrier_lg', 'large_ms', 'fare_lg',
       'carrier_low', 'lf_ms', 'fare_low', 'Geocoded_City1', 'Geocoded_City2',
       'tbl1apk'],
      dtype='object')

In [55]:
# Define flights with relevant variables

flights = flights_data[['airport_1','airport_2','Geocoded_City1','Geocoded_City2','fare','fare_low','fare_lg']]

# Prepare features for Random Forest
X = flights[['airport_1', 'airport_2']]  # You might want to encode these features
y = flights['fare']

# One-hot encoding for categorical variables
X_encoded = pd.get_dummies(X, columns=['airport_1', 'airport_2'], drop_first=True)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict fares for all routes in the dataset
flights['predicted_fare'] = rf_model.predict(X_encoded)

# Define flights_dict using predicted fares
flights_dict = {(row['airport_1'], row['airport_2']): row['predicted_fare'] for index, row in flights.iterrows()}

# Create a linear programming problem
problem = LpProblem("Cheapest_Flight_Problem", LpMinimize)

# Define decision variables based on flights_dict
x = LpVariable.dicts("Flight", flights_dict.keys(), lowBound=0, cat='Binary')

# Objective function: Minimize total fare based on predicted_fare
problem += lpSum(flights_dict[key] * x[key] for key in flights_dict.keys()), "Total_Cost"

# Define your source and destination
source = 'ORD'  # Replace with an actual city code
destination = 'JFK'  # Replace with an actual city code

# Add constraints: at least one flight from source and one flight to destination
problem += lpSum(x[(source, airport_2)] for airport_2 in flights[flights['airport_1'] == source]['airport_2']) >= 1, "At_Least_One_Flight_From_Source"
problem += lpSum(x[(city1, destination)] for city1 in flights[flights['airport_2'] == destination]['airport_1']) >= 1, "At_Least_One_Flight_To_Destination"

# Solve the problem
problem.solve()

# Output the results
print("Status:", LpStatus[problem.status])
for key in flights_dict.keys():
    if x[key].varValue > 0:
        print(f"Flight from {key[0]} to {key[1]} with predicted fare: {flights_dict[key]}")

/var/folders/qv/2tgf4xk15yl2x3t63qygm7y40000gn/T/ipykernel_62673/1599120309.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights['predicted_fare'] = rf_model.predict(X_encoded)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/CDC-datathon/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/qv/2tgf4xk15yl2x3t63qygm7y40000gn/T/5c87d72abd514abb84d8005ffd58f0f0-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/qv/2tgf4xk15yl2x3t63qygm7y40000gn/T/5c87d72abd514abb84d8005ffd58f0f0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 12408 RHS
At line 12411 BOUNDS
At line 16481 ENDATA
Problem MODEL has 2 rows, 4069 columns and 193 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.4794 - 0.00 seconds
Cgl0004I processed model has 2 rows, 192 columns (192 integer (192 of which binary)) and 193 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 174.569
Cbc0038I Before mini branch and bound, 192 integers